In [1]:
import pandas as pd
import numpy as np
import json

In [2]:
matches = pd.read_csv('./data/raw/nba/games.csv').assign(ts = lambda _d: (pd.to_datetime(_d['GAME_DATE_EST']).astype(int) / 10**9).astype(int))
players = pd.read_csv('./data/raw/nba/games_details.csv')

/tmp/ipykernel_5427/111288229.py:2: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  players = pd.read_csv('./data/raw/nba/games_details.csv')


In [3]:
timestamps = matches[['GAME_ID', 'ts']].dropna().drop_duplicates()
matches = matches[['HOME_TEAM_ID', 'VISITOR_TEAM_ID', 'HOME_TEAM_WINS', 'ts']].dropna().drop_duplicates()
players = players.loc[players['COMMENT'].isna(), ['GAME_ID', 'TEAM_ID', 'PLAYER_ID']].dropna().drop_duplicates()
gamelength = 2880

In [4]:
players = (players
           .merge(timestamps, on='GAME_ID', how='inner')
           .rename(columns={'TEAM_ID': 'u', 'PLAYER_ID': 'v'})
           [['u', 'v', 'ts']])

In [5]:
matches = (matches
           .assign(ts = lambda _d: _d['ts'] + gamelength)
           .rename(columns={'HOME_TEAM_ID': 'u', 'VISITOR_TEAM_ID': 'v', 'HOME_TEAM_WINS': 'e_type'})
           .assign(e_type = lambda _d: _d['e_type'] + 1)
           .assign(u_type = 1)
           .assign(v_type = 1)
           .reset_index(drop=True))

players = (players
           .assign(u_type = 1)
           .assign(v_type = 2)
           .assign(e_type = 3)
           .reset_index(drop=True))

In [6]:
teams = pd.concat([matches['u'], matches['v']]).sort_values().unique()
idx_teams = np.arange(len(teams))
teams_dict = {teams[k]: k for k in idx_teams}

player = players['v'].sort_values().unique()
idx_player = np.arange(len(player))
player_dict = {player[k]: k + len(teams) for k in idx_player}

In [7]:
matches = (matches
           .assign(u = lambda _d: _d['u'].map(lambda x: teams_dict[x]))
           .assign(v = lambda _d: _d['v'].map(lambda x: teams_dict[x])))

players = (players
           .assign(u = lambda _d: _d['u'].map(lambda x: teams_dict[x]))
           .assign(v = lambda _d: _d['v'].map(lambda x: player_dict[x])))

In [8]:
events = pd.concat([matches, players]).sort_values('ts').reset_index(drop=True)

In [9]:
NUM_NODE = len(player) + len(teams)
NUM_EV = len(events)
NUM_N_TYPE = 2
NUM_E_TYPE = 3

In [10]:
events = (events
          .assign(e_idx = np.arange(1, NUM_EV + 1)))

In [11]:
print("num node:", NUM_NODE)
print("num events:", NUM_EV)
events.to_csv('./data/processed/nba/events.csv', index=None)
desc = {
        "num_node": NUM_NODE,
        "num_edge": NUM_EV,
        "num_node_type": NUM_N_TYPE,
        "num_edge_type": NUM_E_TYPE
    }
with open('./data/processed/nba/desc.json', 'w') as f:
    json.dump(desc, f, indent=4)

num node: 2641
num events: 585293
